# FreeStep 3D Syringe Pump Calibration Guide

This notebook provides step-by-step instructions for calibrating FreeStep 3D Syringe Pumps in MechWolf. Calibration is essential for accurate flow rate control and reproducible experimental results.

## What You'll Need

- FreeStep 3D Syringe Pump hardware connected via USB
- Syringe(s) for calibration
- Accurate scale for measuring dispensed liquid
- Graduated cylinder or other vessel for collecting liquid
- Water or other suitable calibration fluid

In [ ]:
# Import the necessary modules
import sys
import os
from pathlib import Path

# Add the MechWolf directory to the path if needed
mechwolf_path = Path.home() / "MechWolf_dev"
if mechwolf_path.exists() and str(mechwolf_path) not in sys.path:
    sys.path.append(str(mechwolf_path))

# Import the calibration tool
from mechwolf.components.contrib.calibration_3DSyringePumps_mLmin import JupyterCalibrationTool

## Calibration Process Overview

The calibration process involves several key steps:

1. **Setup and Connection**: Connect your pump hardware and establish a serial connection.
2. **Profile Management**: Create and manage MCU (Microcontroller Unit) and motor profiles.
3. **Association**: Associate your motors with the appropriate MCU.
4. **Calibration**: Run calibration trials to determine flow rate parameters.
5. **Testing**: Verify calibration by testing at different flow rates.

Let's start by launching the calibration tool.

In [ ]:
# Create and display the calibration tool interface
tool = JupyterCalibrationTool()

## Step 1: Connection Setup

First, you need to establish a connection with your FreeStep pump hardware:

1. Ensure your pump is connected via USB to your computer.
2. Go to the **Connection** tab in the interface above.
3. Click **Refresh Ports** to see available serial ports.
4. Select your pump's port from the dropdown (typically COM# on Windows or /dev/tty* on Mac/Linux).
5. Click **Connect** to establish the connection.

The status output will confirm if the connection was successful.

## Step 2: Create MCU Profile

An MCU (Microcontroller Unit) profile represents the Arduino or other microcontroller that controls your pump motors:

1. Go to the **MCU Profiles** tab.
2. Enter a descriptive name for your MCU (e.g., "Arduino Uno Pump Controller").
3. Click **Add MCU Profile** to create it.

You should see your new MCU profile appear in the list below. Each MCU is assigned a unique ID that will be used to identify it in your MechWolf protocols.

## Step 3: Create Motor Profile

Motor profiles represent the individual stepper motors that drive your syringes:

1. Go to the **Motor Profiles** tab.
2. Enter a descriptive name for your motor (e.g., "Syringe 1 Motor").
3. Click **Add Motor Button** to create it.

Next, you need to associate this motor with an MCU and specify which pins control it:

1. Under **Associate Motor with MCU**, select the motor you just created.
2. Select the MCU you created earlier.
3. Enter the step pin number (the Arduino pin connected to the stepper driver's STEP input).
4. Enter the direction pin number (the Arduino pin connected to the stepper driver's DIR input).
5. Click **Associate Motor with MCU**.

The motor will now be listed as configured with your MCU in the "Available Motor Profiles" section.

## Step 4: Perform Calibration

Calibration requires running the pump at two different speeds and measuring the actual dispensed volume:

### Preparation:

1. Fill your syringe with water or another suitable calibration fluid.
2. Mount the syringe securely in your pump.
3. Prepare a vessel to collect and measure the dispensed fluid (pre-weighed container on a scale works best).

### Calibration Steps:

1. Go to the **Calibration** tab.
2. Select the motor you want to calibrate.
3. Enter syringe information (required for accurate calibration):
   - Brand/Manufacturer (e.g., "BD", "Hamilton")
   - Model/Type (e.g., "Plastic", "Glass Precision")
   - Volume in mL (e.g., "10" for a 10mL syringe)
   - Inner Diameter in mm (check the manufacturer specifications)

4. For the **First Calibration Trial**:
   - Set a low frequency (e.g., 100 Hz) and reasonable duration (e.g., 10-30 seconds)
   - Click **Run First Trial** to start dispensing
   - After the run completes, measure the dispensed volume accurately
   - Enter the measured volume (in mL) in the "Volume (mL)" field

5. For the **Second Calibration Trial**:
   - Set a higher frequency (e.g., 500 Hz) and the same duration
   - Click **Run Second Trial** to start dispensing
   - After the run completes, measure the dispensed volume accurately
   - Enter the measured volume (in mL) in the "Volume (mL)" field

6. Click **Calculate Calibration** to generate the calibration parameters

The system will calculate the relationship between motor speed (frequency) and flow rate (mL/min) based on your two data points. The calibration results will be displayed and saved to the motor profile.

## Step 5: Test Your Calibration

After calibration, verify that your pump operates at the expected flow rates:

1. Go to the **Testing** tab.
2. Select the MCU and motor you calibrated.
3. Set a desired flow rate (mL/min) within the calculated min/max range.
4. Choose direction (forward/backward) and duration.
5. Click **Run Test** to start the test.

For advanced testing, you can also:

- Uncheck "Use calibrated syringe data" and enter a custom syringe diameter to see how the system compensates for different syringe sizes.
- Review the "Calibration Details" panel that shows the calculations used to determine the motor speed.

The system will automatically adjust the motor frequency based on your calibration and syringe parameters.

## Tips for Accurate Calibration

1. **Measurement Accuracy**: Use a precise scale (0.01g or better) to weigh dispensed water. 1g of water equals approximately 1mL.

2. **Multiple Trials**: For best results, you can run multiple calibrations and take the average.

3. **Temperature Effects**: Be aware that fluid viscosity can change with temperature, affecting flow rates.

4. **Syringe Specifications**: Always use the exact inner diameter of your syringe. Manufacturer specifications are usually more accurate than manual measurements.

5. **Test Different Flow Rates**: After calibration, verify performance across the entire range of flow rates you plan to use.

6. **Recalibrate Periodically**: Mechanical wear and other factors may change calibration over time. Recalibrate periodically for critical applications.

7. **Different Fluids**: Calibration with water will be most accurate for aqueous solutions. Highly viscous or non-Newtonian fluids may require additional compensation.

## Using Calibrated Pumps in MechWolf Protocols

Once calibrated, you can use your pump in MechWolf protocols. The Pump Code Generator (available in another notebook) can help you generate the initialization code.

When setting up your protocol, you'll need:

1. The MCU ID (displayed in the MCU Profiles tab)
2. The Motor ID (displayed in the Motor Profiles tab)
3. The port name used for connection

Example code for a calibrated FreeStep pump:

```python
from mechwolf import FreeStepPump

# Create a pump with your calibrated profiles
pump = FreeStepPump(
    name="my_pump",
    port="COM3",
    mcu_id="your_mcu_id_here",
    motor_id="your_motor_id_here"
)
```

Then, in your protocol, you can specify flow rates directly in mL/min:

```python
protocol.add([pump, vessel], rate="5 mL/min", duration="2 minutes")
```

## Cleaning Up

When you're done with calibration, run the cleanup code below to properly close connections:

In [ ]:
# Run this cell when you're done with calibration
try:
    tool.cleanup()
    print("Calibration tool cleaned up successfully.")
except NameError:
    print("Calibration tool not initialized. No cleanup required.")

## Troubleshooting

### Connection Issues

- **Port not found**: Make sure the device is plugged in and drivers are installed.
- **Cannot connect**: Try restarting the device, or check if another program is using the port.
- **Arduino not responding**: Verify the correct firmware is installed on your Arduino.

### Calibration Issues

- **Inconsistent measurements**: Check for air bubbles in the syringe or tubing.
- **Motor not moving**: Verify pin assignments and electrical connections.
- **Calculation errors**: Ensure you've entered accurate measurements for both trials.

### Profile Management

- **Cannot find saved profiles**: Profiles are stored in JSON files in the same directory as the calibration module.
- **Changes not saving**: Ensure you click the appropriate 'Save' button after making changes.

For additional support, please check the MechWolf documentation or contact the developers.